In [35]:
import numpy as np
import pandas as pd
import yfinance as yf
import os
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.preprocessing import MinMaxScaler

In [36]:
from data.utils import download_data_yf
from environment.Market import Market
tickers = ["AAPL", "MSFT"]
start_date = "2020-01-01"
end_date = "2024-01-01"



data = download_data_yf(tickers=tickers, start_date=start_date, end_date=end_date)

mkt = Market(data, 2, dt.datetime(2021, 5, 5))

[*********************100%***********************]  2 of 2 completed
c:\Users\m.narese\Desktop\THESIS\REPO\portfolio_optimization\beta\data\utils.py:21: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level=0).reset_index()


In [37]:
mkt.today()

AttributeError: 'Market' object has no attribute 'today'

In [3]:


data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by=['Ticker', 'Date'])

# Group by Ticker and calculate percentage change
data['Pct_Change'] = data.groupby('Ticker')['Close'].pct_change()

# Pivot the DataFrame to get tickers as columns
pivoted_data = data.pivot(index='Date', columns='Ticker', values='Pct_Change')

# Optional: Rename columns to include "_pct_change" for clarity
pivoted_data.columns = [f"{ticker}_pct_change" for ticker in pivoted_data.columns]

# Reset the index if needed
pivoted_data.reset_index(inplace=True)

print(pivoted_data)

          Date  AAPL_pct_change  MSFT_pct_change
0   2010-01-04              NaN              NaN
1   2010-01-05         0.001729         0.000323
2   2010-01-06        -0.015906        -0.006137
3   2010-01-07        -0.001849        -0.010399
4   2010-01-08         0.006648         0.006897
..         ...              ...              ...
499 2011-12-23         0.011993         0.008524
500 2011-12-27         0.007934         0.000384
501 2011-12-28        -0.009569        -0.008448
502 2011-12-29         0.006159         0.007746
503 2011-12-30        -0.000296        -0.002306

[504 rows x 3 columns]


In [4]:
data[1:3]

Price,Date,Ticker,Open,High,Low,Close,Volume,Pct_Change
2,2010-01-05,AAPL,6.465187,6.495012,6.424515,6.458559,601904800,0.001729
4,2010-01-06,AAPL,6.458559,6.484167,6.349199,6.355827,552160000,-0.015906


In [5]:
wf = 30
x_train = []
y_train = []
for company in data["Ticker"].unique():
    data_company = data[data["Ticker"] == company]
    for i in range(wf, len(data_company)):
        x_train.append(data_company[i-wf:i][["Open", "High", "Low", "Close", "Volume"]])
        y_train.append(data_company[i:i+1]["Close"])

x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape x_train to a 3D array with the appropriate dimensions for the LSTM model
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 5))


In [6]:
x_train.shape, y_train.shape

((948, 30, 5), (948, 1))

In [ ]:

def LSTM_model():
    """
    Create and configure an LSTM model for stock price prediction.

    :return: The configured LSTM model (keras.Sequential)
    """

    # Initialize a sequential model
    model = Sequential()

    # Add the first LSTM layer with 50 units, input shape, and return sequences
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 5)))
    # Add dropout to prevent overfitting
    model.add(Dropout(0.2))

    # Add a second LSTM layer with 50 units and return sequences
    model.add(LSTM(units=50, return_sequences=True))
    # Add dropout to prevent overfitting
    model.add(Dropout(0.2))

    # Add a third LSTM layer with 50 units
    model.add(LSTM(units=50))
    # Add dropout to prevent overfitting
    model.add(Dropout(0.2))

    # Add a dense output layer with one unit
    model.add(Dense(units=1))

    return model